# Building AI Agent Bot With RAG, Langchain, and Reasoning Engine From Scratch

## Setup


* This notebook will walk you through some required setup that you need to do before starting with the materials.

* It is highly recommended to use new virtual environment when running jupyter notebook for this workshop.

### Installing dependencies

In [ ]:
%%writefile requirements.txt

google-cloud-aiplatform==1.69.0
google-cloud-aiplatform[langchain]
google-cloud-aiplatform[reasoningengine]
langchain==0.2.16
langchain_core==0.2.39
langchain_community==0.2.17
langchain-google-vertexai==1.0.10
cloudpickle==3.1.0
pydantic==2.9.2
langchain-google-community==1.0.8
google-cloud-discoveryengine==0.12.3
nest-asyncio
asyncio==3.4.3
asyncpg==0.29.0
cloud-sql-python-connector[asyncpg]==1.12.1
langchain-google-cloud-sql-pg==0.11.0
numpy
pandas
pgvector==0.3.5
psycopg2-binary==2.9.9


In [ ]:
!pip install -r requirements.txt

in case you are facing issue with installing psycopg2, please run the following command (linux only):

```
sudo apt update
sudo apt install python3-dev libpq-dev
```

You will require to restart the jupyter kernel once the dependency installed.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Setting up Google Cloud Account

#### Recommended account setup

if you are running this in jupyter notebook locally, you may need to login to google cloud by running the following command from terminal:

```
gcloud auth login
gcloud auth application-default login
```

If you are using Google Colabs, you need to authenticate with your google account by running the following notebook cell. 

> Please remember that you will need to do this on each jupyter notebook during this workshop

In [ ]:
#@markdown ###Authenticate your Google Cloud Account and enable APIs.
# Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

## Accessing Google Cloud Credit

Please redeem your $5 USD credit that you can use for this workshop. Link for this, will be shared on the class room.

The instruction given will also require you to create a new GCP project. Create one!

## Enabling Google Service API

Before creating cloud resources (e.g. database, cloudrun services, reasoning engine, etc), first we must enable the services api.

In [1]:
# @markdown Replace the required placeholder text below. You can modify any other default values, if you like.

# please change the project id into your gcp project id you just created. 
project_id = "imrenagi-gemini-experiment"  # @param {type:"string"}

# you can leave this the same.
region = "us-central1"  # @param {type:"string"}

In [2]:
from googleapiclient import discovery
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=project_id)
response = request.execute()
project_number = response["projectNumber"]
project_number

'896489987664'

Here, we will enable few services:

* `aiplatform.googleapis.com` -> used for using Gemini LLM and reasoning engine
* `run.googleapis.com` -> used for deploying to cloud run
* `cloudbuild.googleapis.com` -> used for building docker image and perform the deployment

In [ ]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable run.googleapis.com 
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable sqladmin.googleapis.com

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:{project_number}-compute@developer.gserviceaccount.com \
    --role="roles/cloudbuild.builds.builder"

# # Add a new role binding to the IAM policy
!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/discoveryengine.editor"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/cloudsql.client"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/run.invoker"

# Deploying Dummy API server

Later on this workshop, you will be using your AI agent to interact with api in order to get detail about an online course you provide as well as to create purchase request. Hence, we will deploy the simple stupid API to cloudrun.

If you want to see the detail, you can check the `api/` directory.

Now let's deploy the Go API to cloud run:

In [ ]:
!echo ${PWD}
!./api/deploy.sh

Once it is deployed, run the command to get the url of your dummy api. Take note because we will use it later:

In [8]:
!gcloud run services describe courses-api --region=us-central1 --format='value(status.url)'

https://courses-api-uzttxm4diq-uc.a.run.app


# Creating Staging Bucket for AI Agent

Later, when we deploy the AI Agent, we have to provide the staging gcs bucket used to store the pickle and some other configurations of our reasoning engine. So, let's create a new empty bucket. Please change `staging_bucket_name` variable below with globally unique name.

Once the bucket created, take note the name of the bucket.

In [ ]:
# change this with globaly unique name. you may add your name to make it unique. this bucket will be used later for storing the model
staging_bucket_name = "courses-imrenagicom-agent"

!gcloud storage buckets create gs://{staging_bucket_name} --project={project_id} --location={region} --uniform-bucket-level-access